In [ ]:
"""
LETA Validation - Additional Analyses (COLAB VERSION)
======================================================

This script performs:
  B1. Transformer vs Self-Report on RW3D
  B2. Text length control (partial correlations)
  B3. Subgroup cross-validation (gender, age splits)
  C.  Per-emotion LETA vs Transformer correlations (Blog corpus)

OUTPUT FILES:
  - transformer_emotions_rw3d.csv  (NEW - transformer predictions on RW3D)
  - results_B1_transformer_vs_selfreport.csv
  - results_B2_text_length_control.csv
  - results_B3_subgroup_validation.csv
  - results_C_per_emotion_leta_vs_transformer.csv

Requirements:
  pip install pandas numpy scipy scikit-learn transformers torch
"""

import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import pearsonr
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURATION - Update paths for your Colab environment
# ============================================================================

# RW3D dataset
RW3D_PATH = "RW3D_dataset.csv"

# Blog corpus files (for Analysis C)
TRANSFORMER_BLOG_PATH = "transformer_emotions_blogs.csv"  # Renamed from transformer_emotions.csv
LETA_BLOG_PATH = "pillar4_post_emotions.csv"

# NRC Emotion Lexicon
NRC_LEXICON_PATH = "NRC-Emotion-Lexicon-Wordlevel-v0.92.txt"

# RW3D column names (verified from your dataset)
TEXT_COLUMN = "text_long_wave1"
SELF_REPORT_COLUMNS = {
    'anger': 'anger_wave1',
    'disgust': 'disgust_wave1',
    'fear': 'fear_wave1',
    'anxiety': 'anxiety_wave1',
    'sadness': 'sadness_wave1',
    'happiness': 'happiness_wave1',
}
GENDER_COLUMN = "sex_wave1"
AGE_COLUMN = "age_wave1"


# ============================================================================
# UTILITY FUNCTIONS
# ============================================================================

def load_nrc_lexicon(path):
    """Load NRC Emotion Lexicon"""
    lexicon = {}
    try:
        with open(path, 'r', encoding='utf-8') as f:
            for line in f:
                parts = line.strip().split('\t')
                if len(parts) == 3:
                    word, emotion, value = parts
                    if int(value) == 1:
                        if word not in lexicon:
                            lexicon[word] = []
                        lexicon[word].append(emotion)
        print(f"✓ Loaded NRC lexicon: {len(lexicon)} words")
    except FileNotFoundError:
        print(f"✗ NRC lexicon not found at {path}")
    return lexicon


def extract_leta_emotions(text, lexicon):
    """Extract LETA emotion scores from text"""
    emotions = {e: 0.0 for e in ['anger', 'anticipation', 'disgust', 'fear',
                                  'joy', 'sadness', 'surprise', 'trust']}

    if pd.isna(text) or not isinstance(text, str):
        return emotions

    words = text.lower().split()
    word_count = len(words)

    if word_count == 0:
        return emotions

    for word in words:
        if word in lexicon:
            for emotion in lexicon[word]:
                if emotion in emotions:
                    emotions[emotion] += 1

    # Normalize by word count
    for e in emotions:
        emotions[e] = emotions[e] / word_count

    return emotions


def partial_correlation(x, y, control):
    """Calculate partial correlation controlling for a third variable"""
    x = np.array(x, dtype=float)
    y = np.array(y, dtype=float)
    control = np.array(control, dtype=float)

    mask = ~(np.isnan(x) | np.isnan(y) | np.isnan(control))
    x, y, control = x[mask], y[mask], control[mask]

    if len(x) < 10:
        return np.nan, np.nan

    # Residualize
    x_resid = x - np.polyval(np.polyfit(control, x, 1), control)
    y_resid = y - np.polyval(np.polyfit(control, y, 1), control)

    return pearsonr(x_resid, y_resid)


# ============================================================================
# ANALYSIS B1: Transformer vs Self-Report on RW3D
# ============================================================================

def run_transformer_on_rw3d(texts):
    """
    Run transformer emotion classifier on RW3D texts
    Creates: transformer_emotions_rw3d.csv
    """
    print("\n" + "="*60)
    print("B1: RUNNING TRANSFORMER ON RW3D")
    print("="*60)

    from transformers import pipeline

    print("Loading DistilRoBERTa-emotion model...")
    classifier = pipeline(
        "text-classification",
        model="j-hartmann/emotion-english-distilroberta-base",
        top_k=None,
        truncation=True,
        max_length=512
    )

    print(f"Processing {len(texts)} texts...")
    results = []

    for idx, text in enumerate(texts):
        if idx % 100 == 0:
            print(f"  {idx}/{len(texts)} ({100*idx/len(texts):.0f}%)")

        if pd.isna(text) or not isinstance(text, str) or len(str(text).strip()) == 0:
            results.append({
                'anger': 0, 'disgust': 0, 'fear': 0,
                'joy': 0, 'sadness': 0, 'surprise': 0, 'neutral': 0
            })
        else:
            try:
                preds = classifier(str(text)[:512])[0]
                result = {p['label']: p['score'] for p in preds}
                results.append(result)
            except Exception as e:
                print(f"  Warning at idx {idx}: {e}")
                results.append({
                    'anger': 0, 'disgust': 0, 'fear': 0,
                    'joy': 0, 'sadness': 0, 'surprise': 0, 'neutral': 0
                })

    transformer_df = pd.DataFrame(results)

    # Save output
    output_file = "transformer_emotions_rw3d.csv"
    transformer_df.to_csv(output_file, index=False)
    print(f"\n✓ Saved: {output_file}")

    return transformer_df


def analyze_transformer_vs_selfreport(rw3d_df, transformer_df):
    """Compare transformer predictions to self-reported emotions"""
    print("\n" + "-"*60)
    print("TRANSFORMER vs SELF-REPORT CORRELATIONS")
    print("-"*60)

    # Mapping: transformer output → RW3D self-report column
    pairs = [
        ('anger', 'anger_wave1'),
        ('disgust', 'disgust_wave1'),
        ('fear', 'fear_wave1'),
        ('fear', 'anxiety_wave1'),  # Also test fear→anxiety
        ('sadness', 'sadness_wave1'),
        ('joy', 'happiness_wave1'),
    ]

    results = []
    for trans_col, human_col in pairs:
        if trans_col in transformer_df.columns and human_col in rw3d_df.columns:
            x = transformer_df[trans_col].values
            y = rw3d_df[human_col].values

            mask = ~(np.isnan(x) | np.isnan(y))
            if mask.sum() > 30:
                r, p = pearsonr(x[mask], y[mask])
                results.append({
                    'transformer': trans_col,
                    'self_report': human_col,
                    'r': round(r, 3),
                    'p': f"{p:.2e}" if p < 0.001 else round(p, 3),
                    'n': int(mask.sum())
                })

    results_df = pd.DataFrame(results)
    print(results_df.to_string(index=False))

    # Summary stats
    mean_r = results_df['r'].mean()
    print(f"\nMean r: {mean_r:.3f}")
    print(f"Range: {results_df['r'].min():.3f} to {results_df['r'].max():.3f}")

    # Critical comparison
    print("\n" + "="*60)
    print("*** CRITICAL COMPARISON ***")
    print("="*60)
    print(f"Transformer vs Self-Report (RW3D): mean r = {mean_r:.3f}")
    print(f"LETA vs Self-Report (RW3D):        mean r = 0.176")
    print()

    if mean_r < 0.25:
        print("→ INTERPRETATION: The expressed-felt gap is UNIVERSAL")
        print("  Both lexicon AND transformer show weak correlation with self-report.")
        print("  This STRENGTHENS the construct boundary argument.")
    elif mean_r < 0.35:
        print("→ INTERPRETATION: Transformer shows MODEST improvement over lexicon")
        print("  Gap persists but is somewhat smaller with contextual methods.")
    else:
        print("→ INTERPRETATION: Transformer SUBSTANTIALLY outperforms lexicon")
        print("  The expressed-felt gap may be more method-specific.")
        print("  Consider reframing paper scope.")

    # Save
    results_df.to_csv("results_B1_transformer_vs_selfreport.csv", index=False)
    print(f"\n✓ Saved: results_B1_transformer_vs_selfreport.csv")

    return results_df


# ============================================================================
# ANALYSIS B2: Text Length Control
# ============================================================================

def analyze_text_length_control(rw3d_df, leta_scores_df):
    """Partial correlations controlling for text length"""
    print("\n" + "="*60)
    print("B2: TEXT LENGTH CONTROL")
    print("="*60)

    # Word counts
    word_counts = rw3d_df[TEXT_COLUMN].apply(
        lambda x: len(str(x).split()) if pd.notna(x) else 0
    ).values

    print(f"Word count: mean={np.mean(word_counts):.0f}, range={np.min(word_counts)}-{np.max(word_counts)}")

    # LETA emotion → self-report pairs
    pairs = [
        ('anger', 'anger_wave1'),
        ('disgust', 'disgust_wave1'),
        ('fear', 'fear_wave1'),
        ('fear', 'anxiety_wave1'),
        ('sadness', 'sadness_wave1'),
        ('joy', 'happiness_wave1'),
    ]

    results = []
    for leta_col, human_col in pairs:
        if leta_col in leta_scores_df.columns and human_col in rw3d_df.columns:
            x = leta_scores_df[leta_col].values
            y = rw3d_df[human_col].values

            # Raw correlation
            mask = ~(np.isnan(x) | np.isnan(y))
            r_raw, p_raw = pearsonr(x[mask], y[mask])

            # Partial correlation (controlling for word count)
            r_partial, p_partial = partial_correlation(x, y, word_counts)

            results.append({
                'leta': leta_col,
                'self_report': human_col,
                'r_raw': round(r_raw, 3),
                'r_partial': round(r_partial, 3) if not np.isnan(r_partial) else 'NA',
                'change': round(r_partial - r_raw, 3) if not np.isnan(r_partial) else 'NA',
                'n': int(mask.sum())
            })

    results_df = pd.DataFrame(results)
    print("\nRaw vs Partial Correlations (controlling for word count):")
    print("-"*60)
    print(results_df.to_string(index=False))

    # Interpret
    changes = [r['change'] for r in results if r['change'] != 'NA']
    if changes:
        mean_change = np.mean(changes)
        print(f"\nMean change after controlling for length: {mean_change:.3f}")
        if abs(mean_change) < 0.02:
            print("→ Text length has MINIMAL impact")
        else:
            print("→ Text length DOES affect correlations")

    results_df.to_csv("results_B2_text_length_control.csv", index=False)
    print(f"\n✓ Saved: results_B2_text_length_control.csv")

    return results_df


# ============================================================================
# ANALYSIS B3: Subgroup Cross-Validation
# ============================================================================

def analyze_subgroups(rw3d_df, leta_scores_df):
    """Test correlation consistency across demographic subgroups"""
    print("\n" + "="*60)
    print("B3: SUBGROUP CROSS-VALIDATION")
    print("="*60)

    results = []

    # Use fear as exemplar emotion
    leta_col = 'fear'
    human_col = 'anxiety_wave1'  # Most relevant pairing

    x = leta_scores_df[leta_col].values
    y = rw3d_df[human_col].values
    mask = ~(np.isnan(x) | np.isnan(y))

    # Overall
    r_overall, _ = pearsonr(x[mask], y[mask])
    results.append({
        'subgroup': 'Overall',
        'n': int(mask.sum()),
        'r': round(r_overall, 3)
    })

    # Gender split
    if GENDER_COLUMN in rw3d_df.columns:
        print(f"\nGender column: {GENDER_COLUMN}")
        for gender in rw3d_df[GENDER_COLUMN].dropna().unique():
            gender_mask = (rw3d_df[GENDER_COLUMN] == gender).values & mask
            if gender_mask.sum() > 30:
                r, _ = pearsonr(x[gender_mask], y[gender_mask])
                results.append({
                    'subgroup': f'Gender: {gender}',
                    'n': int(gender_mask.sum()),
                    'r': round(r, 3)
                })

    # Age split (median)
    if AGE_COLUMN in rw3d_df.columns:
        ages = pd.to_numeric(rw3d_df[AGE_COLUMN], errors='coerce')
        age_median = ages.median()
        print(f"Age median: {age_median}")

        young_mask = (ages <= age_median).values & mask
        old_mask = (ages > age_median).values & mask

        if young_mask.sum() > 30:
            r, _ = pearsonr(x[young_mask], y[young_mask])
            results.append({
                'subgroup': f'Age ≤ {age_median:.0f}',
                'n': int(young_mask.sum()),
                'r': round(r, 3)
            })

        if old_mask.sum() > 30:
            r, _ = pearsonr(x[old_mask], y[old_mask])
            results.append({
                'subgroup': f'Age > {age_median:.0f}',
                'n': int(old_mask.sum()),
                'r': round(r, 3)
            })

    results_df = pd.DataFrame(results)
    print(f"\nSubgroup correlations ({leta_col} vs {human_col}):")
    print("-"*60)
    print(results_df.to_string(index=False))

    # Consistency check
    r_values = results_df['r'].values
    r_range = max(r_values) - min(r_values)
    print(f"\nRange across subgroups: {r_range:.3f}")

    if r_range < 0.10:
        print("→ Correlations are CONSISTENT across subgroups")
    else:
        print("→ Some variation by subgroup")

    results_df.to_csv("results_B3_subgroup_validation.csv", index=False)
    print(f"\n✓ Saved: results_B3_subgroup_validation.csv")

    return results_df


# ============================================================================
# ANALYSIS C: Per-Emotion LETA vs Transformer (Blog Corpus)
# ============================================================================

def analyze_leta_vs_transformer_blog():
    """Per-emotion correlations on blog corpus"""
    print("\n" + "="*60)
    print("C: PER-EMOTION LETA vs TRANSFORMER (BLOG CORPUS)")
    print("="*60)

    try:
        leta_df = pd.read_csv(LETA_BLOG_PATH)
        trans_df = pd.read_csv(TRANSFORMER_BLOG_PATH)
    except FileNotFoundError as e:
        print(f"✗ File not found: {e}")
        return None

    print(f"LETA posts: {len(leta_df)}")
    print(f"Transformer posts: {len(trans_df)}")

    # Align
    min_len = min(len(leta_df), len(trans_df))
    leta_df = leta_df.iloc[:min_len]
    trans_df = trans_df.iloc[:min_len]

    # Find matching columns
    # LETA columns might be: anger, score_anger, etc.
    # Transformer columns: anger, fear, joy, sadness, disgust, surprise

    emotions = ['anger', 'fear', 'joy', 'sadness', 'disgust', 'surprise']

    results = []
    for emotion in emotions:
        # Find LETA column
        leta_col = None
        for possible in [emotion, f'score_{emotion}']:
            if possible in leta_df.columns:
                leta_col = possible
                break

        trans_col = emotion if emotion in trans_df.columns else None

        if leta_col and trans_col:
            x = leta_df[leta_col].values
            y = trans_df[trans_col].values

            mask = ~(np.isnan(x) | np.isnan(y))
            if mask.sum() > 100:
                r, p = pearsonr(x[mask], y[mask])
                results.append({
                    'emotion': emotion,
                    'r': round(r, 3),
                    'p': f"{p:.2e}" if p < 0.001 else round(p, 3),
                    'n': int(mask.sum())
                })

    if not results:
        print("✗ No matching emotion columns found")
        print(f"  LETA columns: {list(leta_df.columns)[:10]}")
        print(f"  Transformer columns: {list(trans_df.columns)}")
        return None

    results_df = pd.DataFrame(results)
    print("\n" + "-"*60)
    print(results_df.to_string(index=False))

    mean_r = results_df['r'].mean()
    std_r = results_df['r'].std()
    print(f"\nMean r: {mean_r:.3f}")
    print(f"SD: {std_r:.3f}")
    print(f"Range: {results_df['r'].min():.3f} to {results_df['r'].max():.3f}")

    print("\n*** FOR VALIDATION 2 TABLE ***")
    print(f"LETA vs Transformer: mean r = {mean_r:.2f} (range {results_df['r'].min():.2f}–{results_df['r'].max():.2f})")

    results_df.to_csv("results_C_per_emotion_leta_vs_transformer.csv", index=False)
    print(f"\n✓ Saved: results_C_per_emotion_leta_vs_transformer.csv")

    return results_df


# ============================================================================
# MAIN
# ============================================================================

def main():
    print("="*60)
    print("LETA VALIDATION - ADDITIONAL ANALYSES")
    print("="*60)

    # -------------------------
    # Load RW3D
    # -------------------------
    print("\nLoading RW3D dataset...")
    try:
        rw3d_df = pd.read_csv(RW3D_PATH)
        print(f"✓ Loaded: {len(rw3d_df)} rows")
    except FileNotFoundError:
        print(f"✗ File not found: {RW3D_PATH}")
        return

    # -------------------------
    # Compute LETA scores on RW3D
    # -------------------------
    print("\nComputing LETA emotion scores on RW3D...")
    lexicon = load_nrc_lexicon(NRC_LEXICON_PATH)

    if not lexicon:
        print("Cannot proceed without NRC lexicon")
        return

    leta_scores = rw3d_df[TEXT_COLUMN].apply(lambda x: extract_leta_emotions(x, lexicon))
    leta_scores_df = pd.DataFrame(leta_scores.tolist())
    print(f"✓ LETA scores computed for {len(leta_scores_df)} texts")

    # -------------------------
    # B2: Text Length Control
    # -------------------------
    b2_results = analyze_text_length_control(rw3d_df, leta_scores_df)

    # -------------------------
    # B3: Subgroup Validation
    # -------------------------
    b3_results = analyze_subgroups(rw3d_df, leta_scores_df)

    # -------------------------
    # C: Per-emotion LETA vs Transformer (Blog)
    # -------------------------
    c_results = analyze_leta_vs_transformer_blog()

    # -------------------------
    # B1: Transformer vs Self-Report (RW3D)
    # -------------------------
    print("\n" + "="*60)
    print("B1: TRANSFORMER ON RW3D")
    print("="*60)
    print("This will run the transformer model on RW3D texts.")
    print("Estimated time: 10-30 minutes for ~1000 texts")

    run_b1 = input("\nRun transformer analysis? (y/n): ").strip().lower()

    if run_b1 == 'y':
        transformer_df = run_transformer_on_rw3d(rw3d_df[TEXT_COLUMN])
        b1_results = analyze_transformer_vs_selfreport(rw3d_df, transformer_df)
    else:
        print("Skipping B1. Run later with transformer_emotions_rw3d.csv")

    # -------------------------
    # Summary
    # -------------------------
    print("\n" + "="*60)
    print("ANALYSIS COMPLETE")
    print("="*60)
    print("""
Output files created:
  - results_B2_text_length_control.csv
  - results_B3_subgroup_validation.csv
  - results_C_per_emotion_leta_vs_transformer.csv
  - transformer_emotions_rw3d.csv (if B1 was run)
  - results_B1_transformer_vs_selfreport.csv (if B1 was run)

Next steps:
  1. Review results
  2. Report back for manuscript revision
    """)


if __name__ == "__main__":
    main()

LETA VALIDATION - ADDITIONAL ANALYSES

Loading RW3D dataset...
✓ Loaded: 1152 rows

Computing LETA emotion scores on RW3D...
✓ Loaded NRC lexicon: 6453 words
✓ LETA scores computed for 1152 texts

B2: TEXT LENGTH CONTROL
Word count: mean=117, range=6-1016

Raw vs Partial Correlations (controlling for word count):
------------------------------------------------------------
   leta     self_report  r_raw  r_partial  change    n
  anger     anger_wave1  0.209      0.212   0.002 1152
disgust   disgust_wave1  0.185      0.186   0.001 1152
   fear      fear_wave1  0.170      0.171   0.001 1152
   fear   anxiety_wave1  0.175      0.176   0.001 1152
sadness   sadness_wave1  0.161      0.165   0.004 1152
    joy happiness_wave1  0.157      0.156  -0.000 1152

Mean change after controlling for length: 0.002
→ Text length has MINIMAL impact

✓ Saved: results_B2_text_length_control.csv

B3: SUBGROUP CROSS-VALIDATION

Gender column: sex_wave1
Age median: 35.0

Subgroup correlations (fear vs anxiet

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/105 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Processing 1152 texts...
  0/1152 (0%)
  100/1152 (9%)
  200/1152 (17%)
  300/1152 (26%)
  400/1152 (35%)
  500/1152 (43%)
  600/1152 (52%)
  700/1152 (61%)
  800/1152 (69%)
  900/1152 (78%)
  1000/1152 (87%)
  1100/1152 (95%)

✓ Saved: transformer_emotions_rw3d.csv

------------------------------------------------------------
TRANSFORMER vs SELF-REPORT CORRELATIONS
------------------------------------------------------------
transformer     self_report     r        p    n
      anger     anger_wave1 0.268 2.06e-20 1152
    disgust   disgust_wave1 0.148 4.17e-07 1152
       fear      fear_wave1 0.243 7.00e-17 1152
       fear   anxiety_wave1 0.285 6.56e-23 1152
    sadness   sadness_wave1 0.103 4.76e-04 1152
        joy happiness_wave1 0.223 1.77e-14 1152

Mean r: 0.212
Range: 0.103 to 0.285

*** CRITICAL COMPARISON ***
Transformer vs Self-Report (RW3D): mean r = 0.212
LETA vs Self-Report (RW3D):        mean r = 0.176

→ INTERPRETATION: The expressed-felt gap is UNIVERSAL
  Both lexico